In [1]:
import numpy as np
import pandas as pd
import random
import torch

Lets dynamically choose type of device to use for our computations. To run on your own GPU one needs to install pytorch and cuda-toolkit.

In [4]:
!nvidia-smi


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
print(torch.cuda.is_available())
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


False


In [3]:
torch.cuda.get_device_properties(device)

RuntimeError: No CUDA GPUs are available

Define the seed for reproductivity:

In [18]:
def set_seed(seed, use_gpu = True):
    """
    Set SEED for PyTorch reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_gpu:
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

SEED = 196
USE_SEED = True
if USE_SEED:
    set_seed(SEED, torch.cuda.is_available())

**Load the images and masks**

In [2]:
# with data augmentation
images = np.load("data/augmented/combined_images.npy")
masks = np.load("data/augmented/combined_masks.npy")

In [32]:
masks[0].shape

(256, 256, 3)

In [49]:
np.set_printoptions(threshold=np.inf)

In [65]:
result = (masks[0][:,:,0] == masks[0][:,:,1]) & (masks[0][:,:,1] == masks[0][:,:,2])

# Alternatively, using numpy logical_and
result = np.logical_and(masks[0][:,:,0] == masks[0][:,:,1], masks[0][:,:,1] == masks[0][:,:,2])

# The result will be a boolean array of shape (256, 256) indicating where the condition is true
print(result.all())

True


In [14]:
np.max(masks[1].shape, axis = 1)

AxisError: axis 1 is out of bounds for array of dimension 1

**Standaritzation**

In [20]:
mean = np.mean(images, axis = (0,1,2)) / 255
std = np.std(images, axis = (0,1,2)) / 255

print("-----  NORMALIZATION VALUES  -----")
print(f"Mean (RGB): {mean}")
print(f"Standard Deviation (RGB): {std}")

-----  NORMALIZATION VALUES  -----
Mean (RGB): [0.56399276 0.33202926 0.2465689 ]
Standard Deviation (RGB): [0.3123053  0.23121322 0.19635625]


In [21]:
import torchvision
import torchvision.transforms as transforms

transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),  # Convert the image to a PyTorch tensor and normalize it between [0, 1]
    torchvision.transforms.Normalize(mean, std)  # Normalize the tensor using the provided mean and standard deviation
])

**Data splitting**



*   Train-test split (70/30)
*   Train-validation split


In [22]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_images, test_images, train_masks, test_masks = train_test_split(
    images, masks, test_size=0.3, random_state=42)

# Print the shapes of the train and test sets
print("Train images shape:", train_images.shape)
print("Test images shape:", test_images.shape)
print("Train masks shape:", train_masks.shape)
print("Test masks shape:", test_masks.shape)

Train images shape: (2800, 256, 256, 3)
Test images shape: (1200, 256, 256, 3)
Train masks shape: (2800, 256, 256, 3)
Test masks shape: (1200, 256, 256, 3)


In [23]:
#Validation set (10%)
training_images, val_images, training_masks, val_masks = train_test_split(
    train_images, train_masks, test_size=0.1, random_state=42)
# Check shape of training images and masks
print("Training Images Shape:", training_images.shape)
print("Training Masks Shape:", training_masks.shape)

# Check shape of validation images and masks
print("Validation Images Shape:", val_images.shape)
print("Validation Masks Shape:", val_masks.shape)

# Check length of training and validation sets
print("Number of Training Examples:", len(training_images))
print("Number of Validation Examples:", len(val_images))

Training Images Shape: (2520, 256, 256, 3)
Training Masks Shape: (2520, 256, 256, 3)
Validation Images Shape: (280, 256, 256, 3)
Validation Masks Shape: (280, 256, 256, 3)
Number of Training Examples: 2520
Number of Validation Examples: 280


In [24]:
from src.preprocess import KvasirDataset

# Create KvasirDataset objects for train, validation and test sets
train_dataset = KvasirDataset(images=train_images, masks=train_masks, transforms=transforms)
val_dataset = KvasirDataset(images=val_images,masks=val_masks, transforms=transforms)
test_dataset = KvasirDataset(images=test_images, masks=test_masks, transforms=transforms) # not sure about transforms in test set


**Define the model**

In [25]:
config = {}
config['name']='First_try'
config['epochs']=10
config['batch_size']=64

config['arch']='NestedUNet'
config['deep_supervision']=True
config['input_channels']=3
config['num_classes']=1
config['early_stopping']=5 # 5 epochs without improving the dice coefficient
#config['input_w']=128
#config['input_h']=128


In [26]:
from torch.utils.data import DataLoader

train_iterator = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_iterator = DataLoader(test_dataset, batch_size=config['batch_size'])
val_iterator = DataLoader(val_dataset,batch_size=config['batch_size'])

In [27]:
from src.main import NestedUNet
from src.utils import count_parameters

model = NestedUNet(config)
print(f"The model has {count_parameters(model):,} trainable parameters.")

The model has 9,279,396 trainable parameters.


In [28]:
model = model.to(device)

In [29]:
from src.loss import BCEDiceLoss
import torch.optim as optim

criterion = BCEDiceLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4) #TODO apply a scheduler lr

In [30]:
from src.train_val_test import train, evaluate
from collections import OrderedDict

log = OrderedDict([
    ('epoch', []),
    ('lr', []),
    ('loss', []),
    ('iou', []),
    ('dice',[]),
    ('val_loss', []),
    ('val_iou', []),
    ('val_dice',[])
])

best_dice = 0
trigger = 0
for epoch in range(config['epochs']):
    print('Epoch [%d/%d]' % (epoch, config['epochs']))

    # train for one epoch
    train_log = train(config, model, train_iterator, criterion, optimizer, config['deep_supervision'], device)
    # evaluate on validation set
    val_log = evaluate(config, model, val_iterator, criterion, config['deep_supervision'], device)

    print('loss %.4f - iou %.4f - dice %.4f - val_loss %.4f - val_iou %.4f - val_dice %.4f'
          % (train_log['loss'], train_log['iou'], train_log['dice_coef'],val_log['loss'], val_log['iou'], val_log['dice_coef']))

    log['epoch'].append(epoch)
    log['lr'].append(config['lr'])
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['dice'].append(train_log['dice_coef'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])
    log['val_dice'].append(val_log['dice_coef'])
    
    pd.DataFrame(log).to_csv('models/%s/log.csv' %
                             config['name'], index=False)

    trigger += 1

    if val_log['dice_coef'] > best_dice:
        torch.save(model.state_dict(), 'models/%s/model.pth' %
                   config['name'])
        best_dice = val_log['dice_coef']
        print("=> saved best model")
        trigger = 0

    # early stopping
    if config['early_stopping'] >= 0 and trigger >= config['early_stopping']:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

Epoch [0/10]


  0%|          | 0/44 [00:00<?, ?it/s]C:\Users\toniv\anaconda3\envs\VisionProj\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 2.25 GiB. GPU  (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 